We are trying to figure out what an appropriate $\Lambda^2$ value to use to generate an $\eta$ to calculate a robust estimate with a small enough bound while spending reasonable compute resources.

In [1]:
import numpy as np
import pandas as pd

# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt
# %matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

In [4]:
def calc_lambda2(eta, confidence=0.95, N=2):
    return (eta**2 - 1) / ((1 - confidence) * eta**2 / (N + 1) - eta)

def calc_cost(pv1, pv2, cell):
    pv1_cost = pv1 * 500 + 2000 if pv1 > 0 else 0
    pv2_cost = pv2 * 500 + 2000 if pv2 > 0 else 0
    cell_cost = cell * 500
    return pv1_cost + pv2_cost + cell_cost

In [6]:
with open('../../results/full_search/load=3482_pv=7989 6423/12_26_16_30_daysinchunk=365_conf=0.95_epsilon=0.19.csv') as file:
    adagrad_df_3482 = pd.read_csv(file, sep=',')
    adagrad_df_3482_0 = adagrad_df_3482[adagrad_df_3482['type'] == 0]

In [11]:
# with open('../../results/full_search/load=6423_pv=7989 6423/12_26_16_31_daysinchunk=365_conf=0.95_epsilon=0.19.csv') as file:
#     adagrad_df_6423 = pd.read_csv(file, sep=',')
#     adagrad_df_6423_0 = adagrad_df_6423[adagrad_df_6423['type'] == 0]

In [16]:
px.scatter_3d(adagrad_df_3482, x='pv1', y='pv2', z='battery', color='type')

In [58]:
def cheby(df, epsilon=0.05, N=2):
    sigma = df[['pv1', 'pv2', 'battery']].to_numpy()
    mu_eta = np.mean(sigma, axis=0)
    print('mu_eta =', mu_eta)

    eta = len(sigma)

    sigma_eta = np.dot(np.transpose(sigma - mu_eta), sigma - mu_eta) / (eta - 1)
    print('sigma_eta', sigma_eta, sep='\n')
    sigma_eta_inv = np.linalg.inv(sigma_eta)
    print('sigma_eta_inv', sigma_eta_inv)
    
    big_lambda_squared = calc_lambda2(eta, epsilon, N)
    print('big_lambda_squared =', big_lambda_squared)
    
    def calc_lambda(xi):
        return (xi - mu_eta) @ sigma_eta_inv @ (xi - mu_eta)
    
    cheby_pts = []
    min_cost_pair = (-1, -1, -1, float('inf'))

    for pv1 in np.arange(mu_eta[0], 15, 0.1):
        for pv2 in np.arange(mu_eta[1], 15, 0.1):
            cell_lo = mu_eta[2]
            cell_hi = 50
            while cell_hi - cell_lo > 0.1:
                cell_mid = (cell_lo + cell_hi) / 2
                
                lmd = calc_lambda([pv1, pv2, cell_mid])
                
                #print(pv1, pv2, cell_mid, lmd)
                
                if lmd > big_lambda_squared:
                    cell_hi = cell_mid
                else:
                    cell_lo = cell_mid
                    
            if (cell_hi > mu_eta[2] + 0.1):
                xi = (pv1, pv2, cell_hi)
                cost = calc_cost(*xi)
                xi_and_cost = (*xi, cost)
                if cost < min_cost_pair[3]:
                    min_cost_pair = xi_and_cost
                cheby_pts.append(xi_and_cost)

    return min_cost_pair, cheby_pts

In [59]:
min_cost_pair, cheby_pts = cheby(adagrad_df_3482_0, epsilon=0.19)

mu_eta = [ 5.78124436  8.37826655 34.46255388]
sigma_eta
[[ 2.59426602 -2.27267162 -0.3625019 ]
 [-2.27267162  2.73207786  1.18438815]
 [-0.3625019   1.18438815  3.03258168]]
sigma_eta_inv [[ 1.81880233  1.70787555 -0.44960644]
 [ 1.70787555  2.04433785 -0.59427299]
 [-0.44960644 -0.59427299  0.50810394]]
big_lambda_squared = 17.364695895565188


In [60]:
min_cost_pair

(5.781244356321839, 8.378266549425286, 40.349789323477914, 31254.650114612516)

In [61]:
cheby_pts_df = pd.DataFrame(cheby_pts)

fig = px.scatter_3d(adagrad_df_3482_0, x='pv1', y='pv2', z='battery')
fig.add_trace(px.scatter_3d(cheby_pts_df, x=0, y=1, z=2, color=3).data[0])
fig.show()